In [33]:
import os
import shutil
import random
from PIL import Image
import torch
from torchvision import transforms
from tqdm import tqdm

# Define paths
base_dir = "../DATA_PREPARE_ATT_02/AffectNet"  # Replace with your dataset directory
output_dir = "AffPreProcessed"

# Define resolutions and transformations
resolution = (260, 260)
transform = transforms.Compose([
    transforms.Resize(resolution),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

# Helper function to create directory structure
def create_dir_structure(base_dir, output_dir):
    for folder in ["train", "val", "test"]:
        for category in os.listdir(os.path.join(base_dir, folder)):
            os.makedirs(os.path.join(output_dir, folder, category), exist_ok=True)

# Function to convert tensor back to a PIL image
def tensor_to_pil(tensor):
    return transforms.ToPILImage()(tensor)

# Function to preprocess and shuffle dataset
def preprocess_and_shuffle(base_dir, output_dir, transform, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    create_dir_structure(base_dir, output_dir)
    for folder in ["train", "val", "test"]:
        for category in os.listdir(os.path.join(base_dir, folder)):
            images = os.listdir(os.path.join(base_dir, folder, category))
            random.shuffle(images)
            
            # Split into train, val, and test
            total = len(images)
            train_split = int(train_ratio * total)
            val_split = train_split + int(val_ratio * total)
            
            for i, img_name in enumerate(tqdm(images, desc=f"Processing {folder}/{category}")):
                img_path = os.path.join(base_dir, folder, category, img_name)
                img = Image.open(img_path).convert('RGB')
                processed_tensor = transform(img)
                processed_img = tensor_to_pil(processed_tensor)  # Convert tensor to PIL image
                
                # Determine target split
                if i < train_split:
                    target_folder = "train"
                elif i < val_split:
                    target_folder = "val"
                else:
                    target_folder = "test"
                
                # Save image with unique ID
                new_name = f"{category}{random.randint(10000000, 99999999)}.png"
                target_path = os.path.join(output_dir, target_folder, category, new_name)
                processed_img.save(target_path)

# Run the preprocessing
preprocess_and_shuffle(base_dir, output_dir, transform)


Processing test/7: 100%|██████████| 400/400 [00:15<00:00, 25.90it/s]
